COMP 215 - LAB 3 Classes (NEO)
----------------
#### Name:
#### Date:

This lab exercise introduces `class` as a means of organizing related data and functions.

**Building on new concepts from lab 2**:
  * a `record` is a related collection of data, with fields for each data value
  * an `API` is an "Application Programmers Interface" defining how a programmer interacts with a system.
  * *f-string* simplifies string formatting operations

**New Python Concepts**:
  * the `class` keyword allows you define a new data `type`, with a set of operations on that data.
  * a `dataclass` simplifies class definition for classes that primarily encapsulate a data structure.

As usual, the first code cell simply imports all the modules we'll be using...

In [35]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

We'll continue working with [Near Earth Object](https://cneos.jpl.nasa.gov/) data
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

Here's a brief review from Lab 2 on how to use it...

### Review: making a query

Here's a query that gets the record for a single NEO that recently passed by.

In [36]:
API_KEY = 'DEMO_KEY'  # substitute your API key here

def get_neos(start_date):
    """ Return a list of NEO for the week starting at start_date """
    url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&api_key={API_KEY}'
    # Fetch last week's NEO feed
    response = requests.request("GET", url, headers={}, data={})
    data = json.loads(response.text)
    print(f'{data["element_count"]} Near Earth Objects found for week of {start_date}')
    return [neo for dated_records in data['near_earth_objects'].values() for neo in dated_records ]

def get_neo(id):
    """ Return a NEO record for the given id """
    url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
    response = requests.request("GET", url, headers={}, data={})
    return json.loads(response.text)

week_start = '2023-01-15'
neos = get_neos(week_start)
assert len(neos) > 0, f'Oh oh!  No NEOs found for {week_start}'
neo = get_neo(neos[-1]['id'])  # get the very latest NEO
pprint(neo)

118 Near Earth Objects found for week of 2023-01-15
{'absolute_magnitude_h': 24.86,
 'close_approach_data': [{'close_approach_date': '1949-07-04',
                          'close_approach_date_full': '1949-Jul-04 22:19',
                          'epoch_date_close_approach': -646710060000,
                          'miss_distance': {'astronomical': '0.419726307',
                                            'kilometers': '62790161.51016609',
                                            'lunar': '163.273533423',
                                            'miles': '39015997.166588442'},
                          'orbiting_body': 'Earth',
                          'relative_velocity': {'kilometers_per_hour': '73509.6739172087',
                                                'kilometers_per_second': '20.4193538659',
                                                'miles_per_hour': '45676.0526626122'}},
                         {'close_approach_date': '1950-08-07',
                        

## Exercise 1:  Define an Asteroid class

Define a simple Asteroid class with some basic state variables for a single NEO.  Your Asteroid class should define at least 4 "state variables:”

    id
    name
    estimated_diameter (m)
    is_potentially_hazardous (bool)

Operations must include:
* `__init__(self, ...)` method to initialize a new Asteroid object with specific data values
* `__str__(self)`, and `__repr__(self)` methods that return nicely formatted string representations of the object.
  
OR...
use a `@dataclass` and it will supply most of that boilerplate code for you!

Write a little code to test your new class.

In [37]:
# Ex.1 your code here
from dataclasses import dataclass

@dataclass
class Asteroid:
    id: str
    name: str
    estimated_diameter: float
    is_potentially_hazardous: bool

    def __str__(self):
        return f'Asteroid {self.name} (ID: {self.id}) - Diameter: {self.estimated_diameter}m, Hazardous: {self.is_potentially_hazardous}'

    def __repr__(self):
        return f'Asteroid(id={self.id}, name={self.name}, estimated_diameter={self.estimated_diameter}, is_potentially_hazardous={self.is_potentially_hazardous})'

# Testing the Asteroid class
asteroid_data = {
    'id': '100',
    'name': 'SampleAsteroid',
    'estimated_diameter': 200,
    'is_potentially_hazardous': True
}

asteroid = Asteroid(**asteroid_data) # Initializing an Asteroid object using the provided data

print(str(asteroid))

print(repr(asteroid))


Asteroid SampleAsteroid (ID: 100) - Diameter: 200m, Hazardous: True
Asteroid(id=100, name=SampleAsteroid, estimated_diameter=200, is_potentially_hazardous=True)


## Exercise 2: Factory method: Asteriod.from_NEO

We want to be able to construct Asteroid objects easily from the record returned from the NEO API.  

Add an "object factory" method to your class...   

    @classmethod
    from_NEO(cls, neo_id):
        ...

that takes the id for a single NEO, fetches the NEO record from API, constructs and returns an Asteroid object representing that NEO.
This kind of method is called a “Factory” because it constructs an object from raw materials.

Write a little code to test your new class.

In [38]:
# Ex. 2 your code here
from dataclasses import dataclass

@dataclass
class Asteroid:
    id: str
    name: str
    estimated_diameter: float
    is_potentially_hazardous: bool

    def __str__(self):
        return f'Asteroid {self.name} (ID: {self.id}) - Diameter: {self.estimated_diameter}m, Hazardous: {self.is_potentially_hazardous}'

    def __repr__(self):
        return f'Asteroid(id={self.id}, name={self.name}, estimated_diameter={self.estimated_diameter}, is_potentially_hazardous={self.is_potentially_hazardous})'

    @classmethod
    def object_factory(cls, neo_id):
        url = f'https://api.nasa.gov/neo/rest/v1/neo/{neo_id}?api_key={API_KEY}'
        response = requests.get(url)
        neo_data = json.loads(response.text)

        asteroid_data = {
            'id': neo_data['id'],
            'name': neo_data['name'],
            'estimated_diameter': neo_data['estimated_diameter']['meters']['estimated_diameter_max'],
            'is_potentially_hazardous': neo_data['is_potentially_hazardous_asteroid']
        }

        return cls(**asteroid_data) #asteroid object

# Testing the Asteroid class with the factory method
neo_id = '3542519'
asteroid_from_factory = Asteroid.object_factory(neo_id)

print(str(asteroid_from_factory))

print(repr(asteroid_from_factory))


Asteroid (2010 PK9) (ID: 3542519) - Diameter: 258.2497910326m, Hazardous: True
Asteroid(id=3542519, name=(2010 PK9), estimated_diameter=258.2497910326, is_potentially_hazardous=True)


## Exercise 3: Define a CloseApproach class

Each NEO record comes with a list of `close_approach_data`, where each record in this list represents a single “close approach” to another orbiting body.
* Develop a class named `CloseApproach` to represent a single close approach record.
* State variables are

        neo (refrence to related NEO object)
        orbiting body (str)
        approach date (datetime object!)
        miss distance (float - choose units, document it, and be consistent!)
        relative velocity (ditto)
  
Define a "Factory" class method to construct a `CloseApproach` object from one close approach data record (a dictionary object).   
This method takes a `neo` object as input for the NEO to which the close approach data belongs.
Remember to parse the date/time string into a datetime object.

In [39]:
# Ex. 3 your code here
import datetime
import json
import requests
from dataclasses import dataclass

#Asteroid class
@dataclass
class Asteroid:
    id: str
    name: str
    estimated_diameter: float
    is_potentially_hazardous: bool

    def __str__(self):
        return f'Asteroid {self.name} (ID: {self.id}) - Diameter: {self.estimated_diameter}m, Hazardous: {self.is_potentially_hazardous}'

    def __repr__(self):
        return f'Asteroid(id={self.id}, name={self.name}, estimated_diameter={self.estimated_diameter}, is_potentially_hazardous={self.is_potentially_hazardous})'

#CloseApproach Class
class CloseApproach:
    def __init__(self, neo, orbiting_body, approach_date, miss_distance, relative_velocity):
        self.neo = neo
        self.orbiting_body = orbiting_body
        self.approach_date = approach_date
        self.miss_distance = miss_distance
        self.relative_velocity = relative_velocity

    def __str__(self):
        return f"CloseApproach to {self.orbiting_body} on {self.approach_date} - Miss Distance: {self.miss_distance}, Relative Velocity: {self.relative_velocity}"

    def __repr__(self):
        return f"CloseApproach(neo={repr(self.neo)}, orbiting_body={self.orbiting_body}, approach_date={repr(self.approach_date)}, miss_distance={self.miss_distance}, relative_velocity={self.relative_velocity})"

    @classmethod
    def from_close_approach_data(cls, neo, close_approach_data):
        approach_date_str = close_approach_data['close_approach_date']

        approach_date = datetime.datetime.strptime(approach_date_str, "%Y-%m-%d")

        return cls(
            neo=neo,
            orbiting_body=close_approach_data['orbiting_body'],
            approach_date=approach_date,
            miss_distance=float(close_approach_data['miss_distance']),
            relative_velocity=float(close_approach_data['relative_velocity'])
        )

asteroid_data = {
    'id': '100',
    'name': 'SampleAsteroid',
    'estimated_diameter': 200,
    'is_potentially_hazardous': True
}
asteroid = Asteroid(**asteroid_data)

approach_data = {
    'close_approach_date': '2023-02-01',
    'orbiting_body': 'Earth',
    'miss_distance': '100000',
    'relative_velocity': '20000'
}
close_approach_instance = CloseApproach.from_close_approach_data(asteroid, approach_data)

print(str(close_approach_instance))

print(repr(close_approach_instance))


CloseApproach to Earth on 2023-02-01 00:00:00 - Miss Distance: 100000.0, Relative Velocity: 20000.0
CloseApproach(neo=Asteroid(id=100, name=SampleAsteroid, estimated_diameter=200, is_potentially_hazardous=True), orbiting_body=Earth, approach_date=datetime.datetime(2023, 2, 1, 0, 0), miss_distance=100000.0, relative_velocity=20000.0)


## Exercise 4: Add list of CloseApproach objects to the Asteroid

Every `Asteroid` should have a list of “close approaches”.
But there is a catch-22 here because we need the `Asteroid` to construct the `CloseApproach` objects.
Add an instance variable to your Asteroid class with a default value of an empty list:

    ...
    close_approaches:list = []
    ...
      
In `from_NEO` factory, use a list comprehension to build the list of `CloseApproach` objects for the Asteroid instance, and then set the instance's `close_approaches` variable before returning it.  
Setting the value of an object's instance variables from outside the class is generally frowned upon - this is why we make the factory a method of the class itself!

Now add a method to `Asteroid` to return the `nearest_miss` `CloseApproach` object for the asteroic:

    def nearest_miss(self):
        ...

Extend your test code to demonstrate these new features.

In [41]:
import datetime
import json
import requests
from dataclasses import dataclass, field
from typing import List

#CloseApproach class
@dataclass
class CloseApproach:
    neo: 'Asteroid'  # Reference to the Asteroid class
    orbiting_body: str
    approach_date: datetime.datetime
    miss_distance: float
    relative_velocity: float

    @classmethod
    def from_close_approach_data(cls, neo, approach_data):
        return cls(
            neo=neo,
            orbiting_body=approach_data['orbiting_body'],
            approach_date=datetime.datetime.strptime(approach_data['close_approach_date'], "%Y-%m-%d"),
            miss_distance=float(approach_data['miss_distance']['kilometers']),
            relative_velocity=float(approach_data['relative_velocity']['kilometers_per_second'])
        )

#Asteroid class
@dataclass
class Asteroid:
    id: str
    name: str
    estimated_diameter: float
    is_potentially_hazardous: bool
    close_approaches: List[CloseApproach] = field(default_factory=list)

    @classmethod
    def object_factory(cls, neo_id, api_key):

        url = f'https://api.nasa.gov/neo/rest/v1/neo/{neo_id}?api_key={api_key}'
        response = requests.get(url)
        neo_data = json.loads(response.text)

        asteroid_data = {
            'id': neo_data['id'],
            'name': neo_data['name'],
            'estimated_diameter': neo_data['estimated_diameter']['meters']['estimated_diameter_max'],
            'is_potentially_hazardous': neo_data['is_potentially_hazardous_asteroid']
        }

        # Asteroid object
        asteroid = cls(**asteroid_data)

        asteroid.close_approaches = [CloseApproach.from_close_approach_data(asteroid, approach_data) for approach_data in neo_data.get('close_approach_data', [])]

        return asteroid

    def nearest_miss(self):
        if not self.close_approaches:
            return None

        #CloseApproach with the nearest miss distance
        nearest_approach = min(self.close_approaches, key=lambda approach: approach.miss_distance)
        return nearest_approach

API_KEY = 'RtsajZdwuz4dc09kxKhxliof8ztq0e4bXsdiYdzB'

# Testing the Asteroid class with the factory method
neo_id = '3542519'
asteroid_from_factory = Asteroid.object_factory(neo_id, API_KEY)

print(str(asteroid_from_factory))

print(repr(asteroid_from_factory))

# Nearest miss
nearest_miss = asteroid_from_factory.nearest_miss()
if nearest_miss:
    print(f"\nNearest Miss:\n{nearest_miss}")
else:
    print("\nNo close approaches recorded.")


Asteroid(id='3542519', name='(2010 PK9)', estimated_diameter=258.2497910326, is_potentially_hazardous=True, close_approaches=[CloseApproach(neo=..., orbiting_body='Merc', approach_date=datetime.datetime(1900, 6, 1, 0, 0), miss_distance=6664518.761844655, relative_velocity=30.9354328365), CloseApproach(neo=..., orbiting_body='Venus', approach_date=datetime.datetime(1900, 7, 7, 0, 0), miss_distance=21193325.607973628, relative_velocity=31.7784421852), CloseApproach(neo=..., orbiting_body='Earth', approach_date=datetime.datetime(1900, 7, 27, 0, 0), miss_distance=32572329.524414185, relative_velocity=23.864674044), CloseApproach(neo=..., orbiting_body='Merc', approach_date=datetime.datetime(1902, 2, 11, 0, 0), miss_distance=13009477.489399606, relative_velocity=28.1657402294), CloseApproach(neo=..., orbiting_body='Earth', approach_date=datetime.datetime(1903, 9, 2, 0, 0), miss_distance=28729826.208952066, relative_velocity=20.7057263268), CloseApproach(neo=..., orbiting_body='Earth', appro

## Challenge - Take your skills to the next level...
### Exercise 5: add one additional analysis

 With these data structures in place, we can now start answering all kinds of interesting questions about a single Asteroid or a set of Asteroids.  
Here’s a couple ideas to try:

* add a method to the Asteroid class, `closest_earth_approach`, that returns the CloseApproach object that represents the closest approach the Asteroid makes to Earth.

* write a **function** named `most_dangerous_approach`, that takes a date range and returns a single “potentially hazardous” Asteroid object that makes the closest approach to Earth in within that range.  Your algorithm will ultimately need to:
    * grab the list of NEO’s for the given date range;
    * use a list comprehension to build the list of Asteroid objects for the NEO’s returned
    * use a list comprehension to filter  potentially hazardous Asteroids only;
    * use a list comprehension to map each Asteroid to its  closest_earth_approach
    * apply Python’s min function to identify the Asteroid with the closest_earth_approach

You may want to decompose some of these steps into smaller functions.
* add a method to the Asteroid class, estimated_mass, that computes an estimate of the Asteroid’s mass based on its diameter.  This is a model – state your assumptions.
* add a method to the CloseApproach class, impact_force,  that estimates the force of impact if the Asteroid hit the orbiting object.  Again, this is a model, state your assumptions.

In [ ]:
# Ex. 5 (challenge) your code here